# 9. Collaborative Filtering - k-NN

In [1]:
%run "6.Recommendations_Sample.ipynb"

Number of users 5,000
First five users id [ 9880850 11308871 10339131  8414394  9734353]
Number of Rows: 12,663,134
Numbeer of Columns: 37,420
Number of stored values: 47,967,516
Number of Rows: 4,916
Numbeer of Columns: 37,420
Number of stored values: 191,000
Number of Rows: 4,917
Numbeer of Columns: 17,109
Number of stored values: 208,108


In [61]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
import scipy as sp

In [50]:
# Important to remove the user_id and game_id
user_game_matrix_k_rec = user_game_matrix_k_rec[1:,1:]

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(user_game_matrix_k_rec)

In [35]:
similarities.shape

(4916, 4916)

In [41]:
similarities

array([[1.        , 0.        , 0.03031695, ..., 0.        , 0.        ,
        0.10314212],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.0326164 ],
       [0.03031695, 0.        , 1.        , ..., 0.03742406, 0.03300492,
        0.10006256],
       ...,
       [0.        , 0.        , 0.03742406, ..., 1.        , 0.        ,
        0.09549105],
       [0.        , 0.        , 0.03300492, ..., 0.        , 1.        ,
        0.08421519],
       [0.10314212, 0.0326164 , 0.10006256, ..., 0.09549105, 0.08421519,
        1.        ]])

In [48]:
# Choose the Number of neighbours
K = 5

In [49]:
from sklearn.neighbors import NearestNeighbors

# Create an instance of the NearestNeighbors model
knn_model = NearestNeighbors(n_neighbors=10, metric='cosine')

# Fit the model with the cosine similarity matrix
knn_model.fit(similarities)

# Example: Query the nearest neighbors for a specific user
user_index = 0  # Index of the user in the similarity matrix
query_user = similarities[user_index].reshape(1, -1)  # Reshape to match the input format

# Find the k nearest neighbors for the query user
distances, neighbor_indices = knn_model.kneighbors(query_user, K)

# Print the indices of the nearest neighbors and their corresponding distances
print("Nearest neighbors indices:", neighbor_indices)
print("Distances:", distances)

Nearest neighbors indices: [[   0 4602 4397 4148 3957]]
Distances: [[0.         0.31053824 0.3240523  0.32507665 0.33109286]]


In [51]:
# Create an empty table to store the nearest neighbors' indices and distances for all users
neighbor_table = []

# Iterate over each user in the similarity matrix
for user_index in range(similarities.shape[0]):
    # Query the nearest neighbors for the current user
    query_user = similarities[user_index].reshape(1, -1)  # Reshape to match the input format
    distances, neighbor_indices = knn_model.kneighbors(query_user, K)
    
    # Append the user's nearest neighbors' indices and distances to the table
    neighbor_table.append({"User": user_index, "Nearest Neighbors": neighbor_indices[0], "Distances": distances[0]})

# Convert the table to a pandas DataFrame for easier visualization
import pandas as pd
neighbor_table_df = pd.DataFrame(neighbor_table)

# Print the table
print(neighbor_table_df)

      User               Nearest Neighbors  \
0        0     [0, 4602, 4397, 4148, 3957]   
1        1     [1, 2681, 2118, 2553, 2855]   
2        2      [2, 691, 2839, 2160, 4417]   
3        3       [3, 50, 4613, 2719, 3817]   
4        4     [4, 1581, 4740, 1279, 1091]   
...    ...                             ...   
4911  4911     [4911, 4602, 1654, 3549, 9]   
4912  4912    [4912, 10, 2738, 2533, 4196]   
4913  4913   [4913, 481, 3137, 1611, 1975]   
4914  4914  [4914, 2091, 3794, 2907, 1542]   
4915  4915  [4915, 1360, 3710, 3314, 2533]   

                                              Distances  
0     [0.0, 0.3105382391832817, 0.3240522989282518, ...  
1     [0.0, 0.6196120217854968, 0.621864326035992, 0...  
2     [3.9968028886505635e-15, 0.192985555570923, 0....  
3     [2.7755575615628914e-15, 0.3201472887067359, 0...  
4     [8.881784197001252e-16, 0.36653183634780606, 0...  
...                                                 ...  
4911  [0.0, 0.21374393506230105, 0.219444

In [52]:
neighbor_table_df

,User,Nearest Neighbors,Distances
0,0,"[0, 4602, 4397, 4148, 3957]","[0.0, 0.3105382391832817, 0.3240522989282518, ..."
1,1,"[1, 2681, 2118, 2553, 2855]","[0.0, 0.6196120217854968, 0.621864326035992, 0..."
2,2,"[2, 691, 2839, 2160, 4417]","[3.9968028886505635e-15, 0.192985555570923, 0...."
3,3,"[3, 50, 4613, 2719, 3817]","[2.7755575615628914e-15, 0.3201472887067359, 0..."
4,4,"[4, 1581, 4740, 1279, 1091]","[8.881784197001252e-16, 0.36653183634780606, 0..."
...,...,...,...
4911,4911,"[4911, 4602, 1654, 3549, 9]","[0.0, 0.21374393506230105, 0.21944477095132797..."
4912,4912,"[4912, 10, 2738, 2533, 4196]","[3.774758283725532e-15, 0.22145656342369047, 0..."
4913,4913,"[4913, 481, 3137, 1611, 1975]","[0.0, 0.3259784266536667, 0.34419043389376935,..."
4914,4914,"[4914, 2091, 3794, 2907, 1542]","[3.1086244689504383e-15, 0.2058472522181929, 0..."


In [1]:
neighbor_table_df[neighbor_table_df["User"]==2].explode(["Nearest Neighbors","Distances"])

NameError: name 'neighbor_table_df' is not defined

In [77]:
%run "Get_Training_and_Test_Instances.ipynb"

x_test.shape (6533, 2)
y_test.shape (6533, 1)
Number of Users: 4,914
Number of Games: 17,106
x_train.shape (922979, 2)
y_train.shape (922979, 2)


In [101]:
%run "Model_Evaluation.ipynb"

In [121]:
x = get_y_true(2681, x_train, y_train)
x

array([ 1212,  1668,  2177,  3341,  3952,  4772,  4993,  5006,  5460,
        5523,  5584,  6280,  6784,  9519, 10112, 10456, 10585, 10743,
       11714, 12933, 13361, 14352])

In [122]:
y = get_y_true(0, x_train, y_train)
y

array([    4,     7,     8,    18,    20,   251,   382,   496,   664,
        1099,  1365,  1600,  1813,  1985,  2333,  2596,  2656,  2864,
        3061,  3638,  4855,  4871,  4893,  6160,  6881,  6964,  7550,
        7745,  8050, 12610, 14249, 15159, 15699, 15739])

In [123]:
set(x).intersection(set(y))

set()

In [100]:
# Use the mask to filter the rows
filtered_array = x_train[(x_train[:, 0] == 4602)]
filtered_array

array([[ 4602,   229],
       [ 4602,   338],
       [ 4602,   433],
       [ 4602,   590],
       [ 4602,   886],
       [ 4602,   917],
       [ 4602,  1030],
       [ 4602,  1525],
       [ 4602,  1694],
       [ 4602,  1956],
       [ 4602,  1985],
       [ 4602,  1999],
       [ 4602,  2066],
       [ 4602,  2166],
       [ 4602,  2341],
       [ 4602,  2352],
       [ 4602,  2385],
       [ 4602,  2438],
       [ 4602,  2561],
       [ 4602,  2595],
       [ 4602,  3778],
       [ 4602,  3896],
       [ 4602,  4137],
       [ 4602,  4247],
       [ 4602,  4265],
       [ 4602,  4269],
       [ 4602,  4276],
       [ 4602,  4309],
       [ 4602,  4328],
       [ 4602,  4338],
       [ 4602,  4379],
       [ 4602,  4389],
       [ 4602,  4502],
       [ 4602,  4952],
       [ 4602,  5073],
       [ 4602,  5215],
       [ 4602,  5253],
       [ 4602,  5288],
       [ 4602,  5364],
       [ 4602,  5369],
       [ 4602,  6085],
       [ 4602,  6146],
       [ 4602,  6599],
       [ 46

In [103]:
recommendations_sample[recommendations_sample["user_id_categorical"]==users_mapping[4602]]

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,user_id_categorical,app_id_categorical
384223,1174180,2,0,2021-12-31,1,63.9000,13415210,384223,11864372,23871
6923898,335300,2,0,2022-01-23,1,87.7000,13415210,6923898,11864372,4178
983267,1240440,0,0,2021-11-25,1,3.3000,13415210,983267,11864372,25117
9507660,10090,122,49,2021-11-26,1,16.7000,13415210,9507660,11864372,261
19990871,620,0,0,2022-12-24,1,11.6000,13415210,19990871,11864372,23
...,...,...,...,...,...,...,...,...,...,...
33500881,418370,0,0,2021-05-24,1,11.5000,13415210,33500881,11864372,6816
20413215,641990,0,0,2019-11-03,1,55.2000,13415210,20413215,11864372,12616
31318400,677620,0,0,2021-12-31,1,1.0000,13415210,31318400,11864372,13484
3979754,4000,332,244,2021-12-04,1,36.4000,13415210,3979754,11864372,133


In [70]:
users_mapping[4607]

11870990